# **Import Libraries, Data set**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Formatting, Handling text
import string
import re

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS
from collections import Counter

# Split Dataset
from sklearn.model_selection import train_test_split

# Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import optimizers, layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2, l1_l2

# evaluation
from sklearn import metrics
from functools import reduce
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

# **EDA**

In [ ]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
train_data.head().style.background_gradient(cmap='coolwarm')

In [ ]:
print("Number of rows is = ", train_data.shape[0], "\nNumber of columns is = ", train_data.shape[1])

In [ ]:
# keyword, location에서 결측치가 발생함을 알 수 있음.
# Location의 경우, 사용자가 입력하고 자동으로 생성되지 않기 때문에 사용 X

train_data.info()

In [ ]:
# 빨간 색은 null값, text와 target만 사용할 것이므로 따로 처리하지 않음.
sns.set(rc={'figure.figsize':(11,8)})
sns.heatmap(train_data.isnull(), yticklabels=False, cbar=False, cmap="coolwarm")

In [ ]:
train_data['target'].value_counts()

In [ ]:
plt.figure(figsize=(5,3))
colors = ["blue", "red"]

sns.countplot(x='target', data=train_data, palette=colors)
plt.title('Target Distributions \n (0: Non Disaster, 1: Disaster)', fontsize=14)

In [ ]:
# 데이터 고유값(keyword)의 총 갯수 : 총 221개의 키워드가 존재함을 알 수 있음
train_data["keyword"].nunique()

In [ ]:
# 가장 많이 쓰인 20개의 키워드를 표시함.

chains = train_data['keyword'].value_counts()[:20]
sns.barplot(x=chains, y=chains.index, palette='deep')
plt.title("Top 20 Keywords")
plt.xlabel("Count of Keyords")

In [ ]:
# 가장 많이 쓰인 20개의 keyword를 disaster와 non-disaster로 나눠서 시각화함.

disaster_keywords = train_data.loc[train_data["target"]==1]["keyword"].value_counts()
nondisaster_keywords = train_data.loc[train_data["target"]==0]["keyword"].value_counts()

fig, ax = plt.subplots(1, 2, figsize=(20,8))
sns.barplot(y=disaster_keywords[0:20].index, x=disaster_keywords[0:20], orient='h', ax=ax[0], palette="Reds_d")
ax[0].set_title("Top 20 Keywords - Disaster Tweets")
ax[0].set_xlabel("Keyword Frequency")

sns.barplot(y=nondisaster_keywords[0:20].index, x=nondisaster_keywords[0:20], orient='h', ax=ax[1], palette="Blues_d")
ax[1].set_title("Top 20 Keywords - Non-Disaster Tweets")
ax[1].set_xlabel("Keyword Frequency")

plt.tight_layout()
plt.show()

In [ ]:
top_disaster_keyword = train_data.groupby('keyword').mean()['target'].sort_values(ascending=False).head(20)
top_nondisaster_keyword = train_data.groupby('keyword').mean()['target'].sort_values().head(20)

In [ ]:
# 빨간색 : 가장 많이 쓰인 Disaster Keywords
# 파란색 : 가장 적게 쓰인 Non-Disaster Keywords

fig, ax = plt.subplots(1, 2, figsize=(20, 8))

sns.barplot(y=top_disaster_keyword[0:20].index, x=disaster_keywords[0:20], orient='h', ax=ax[0], palette="Reds_d")
ax[0].set_title("Top 20 Keywords - Highest used Disaster Keyword")
ax[0].set_xlabel("Keyword Frequency")

sns.barplot(y=top_nondisaster_keyword[0:20].index, x=top_nondisaster_keyword[0:20], orient='h', ax=ax[1], palette="Blues_d")
ax[1].set_title("Top 20 Keywords - Least used Non-Disaster Tweets")
ax[1].set_xlabel("Keyword Frequency")


plt.tight_layout()
plt.show()

In [ ]:
# 가운데에 있을수록 분류하는데 영향을 미치지 않는 keyword임.

train_data['target_mean'] = train_data.groupby('keyword')['target'].transform('mean')

fig = plt.figure(figsize=(8, 50), dpi=70)

sns.countplot(y=train_data.sort_values(by='target_mean', ascending=False)['keyword'],
              hue=train_data.sort_values(by='target_mean', ascending=False)['target'])

plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=12)
plt.legend(loc=1)
plt.title('Target Distribution in Keywords')

plt.show()

train_data.drop(columns=['target_mean'], inplace=True)

In [ ]:
# location 리스트에 위치 내림차순으로 저장
locations = train_data["location"].value_counts()

In [ ]:

plt.figure(figsize=(10, 7))

sns.barplot(y=locations[0:20].index, x=locations[0:20], orient="h")

plt.title("Top 20 Locations")
plt.show()

In [ ]:
print(len(train_data['text']))

In [ ]:
train_data["length"] = train_data["text"].apply(len)
train_data.head()

In [ ]:
train_data['length'].describe()

In [ ]:
train_data[train_data['length']==157]['text'].iloc[0]

In [ ]:
plt.figure(figsize=(5, 3))
sns.histplot(train_data["length"], kde=True, color="purple", bins=30)
plt.title("length of tweets")
plt.xlabel("Number of Characters")
plt.ylabel("Density")
plt.show()

In [ ]:
train_data.hist(column="length", by='target', bins=60, figsize=(10,3))

In [ ]:
def count_words(x):
    return len(x.split())

train_data["num_words"] = train_data["text"].apply(count_words)

plt.figure(figsize=(5, 3))
sns.histplot(train_data["num_words"], kde=True, color="purple", bins=30)
plt.title("Histogram of Number of Words per Tweet")
plt.xlabel("Number of Words")
plt.ylabel("Density")
plt.show()

In [ ]:
train_data.hist(column='num_words', by='target', bins=60, figsize=(10,3))

# **Preprocessing**

In [ ]:
# Text Refining

In [ ]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
# 숫자 URL은 원래 NUMBER, URL로 치환해서 사용했었으나,
# 긍정 부정 두 경우 다 많이 쓰여서 없애버림

def remove_URL(text):
    url = re.compile(r'https?://\S+|www.\.\S+')
    return url.sub(r'', text)

def remove_HTML(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_not_ASCII(text):
    text = ''.join([word for word in text if word in string.printable])
    return text

def word_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

def replace_abbrev(text):
    string = ""
    for word in text.split():
        string += word_abbrev(word) + " "
    return string

def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'', text)

def remove_punctuation(text):
    clean_text = [char for char in text if char not in string.punctuation]
    clean_text = ''.join(clean_text)
    
    return clean_text

In [ ]:
def clean_txt(text):
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_not_ASCII(text)
    text = replace_abbrev(text)
    text = remove_number(text)
    text = remove_punctuation(text)
    
    return text

In [ ]:
train_data['clean_text'] = train_data["text"].apply(clean_txt)

In [ ]:
# remove stopwords
print(stopwords.words('english'))

In [ ]:
def to_remove_stopword(text):
    remove_stopword = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return remove_stopword

In [ ]:
train_data['clean_text'] = train_data['clean_text'].apply(to_remove_stopword)
train_data.head()

In [ ]:
# 단어와 길이 분포 확인, 시각화

negative_words = np.hstack(train_data[train_data['target']==0]['clean_text'].values)
positive_words = np.hstack(train_data[train_data['target']==1]['clean_text'].values)

positive_word_count = Counter(positive_words)
print(positive_word_count.most_common(20))

negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

In [ ]:
# 트윗의 평균 길이 분포 조사 : 유의미한 차이가 없음.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Words in Texts')

text_len = train_data[train_data['target']==1]['clean_text'].map(lambda x: len(x))
ax1.hist(text_len, color='blue')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('재난 트윗의 평균 길이 :', np.mean(text_len))

text_len = train_data[train_data['target']==0]['clean_text'].map(lambda x: len(x))
ax2.hist(text_len, color='red')
ax2.set_title('Negative Reviews')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('재난X 트윗의 평균 길이 :', np.mean(text_len))

In [ ]:
# Integer Encoding

In [ ]:
# 정수 인코딩을 위해 Vocaburary를 먼저 만들어야 함.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['clean_text'])

In [ ]:
# 정수 인코딩을 통해 고유한 정수를 부여함.
# 등장 횟수가 1회인 단어는 무의미하므로 배제함.

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수 총 합

# 단어와 빈도수의 pair를 key, value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    
    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + 1
        
print("vocab size :", total_cnt)
print("등장 빈도가 %s번 이하인 단어의 수 : %s" %(threshold-1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율 :", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어의 등장 빈도 :", (rare_freq / total_freq)*100)

In [ ]:
# vocab에서 빈도수가 2 이하인 단어는 모두 제거
# 0번은 패딩 토큰, 1번은 OOV 토큰이므로 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :', vocab_size)

In [ ]:
# vocab을 토크나이저의 인자로 넘겨주면, 토크나이저는 텍스트 시퀀스를 숫자로 변환함.

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(train_data['clean_text'])
X = tokenizer.texts_to_sequences(train_data['clean_text'].values)
print(X[:3])

In [ ]:
# Padding을 위해 데이터 길이의 분포 확인
# 최대 길이가 23이므로 넉넉하게 30으로 잡고 할 예정

print('트윗의 최대 길이 :', max(len(tweet) for tweet in X))
print('트윗의 평균 길이 :', sum(map(len, X))/len(X))
plt.hist([len(tweet) for tweet in X], bins=50)
plt.xlabel('length of samples')
plt.ylabel('numbers of samples')
plt.show()

In [ ]:
max_len = 30
X = pad_sequences(X, maxlen=max_len)
print(X[0])
print(X.shape)

In [ ]:
# 가공 이전 문장 : Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
tokenizer.sequences_to_texts([[4217, 714, 152, 56, 1430, 4218, 13]])

# **LSTM Model Creation**

In [ ]:
y = train_data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 41)

In [ ]:
# hyperparameter

embed_dim = 64
lstm_out = 64

In [ ]:
def build_hybrid_model():
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,), name="input"))
    model.add(layers.Embedding(input_dim=num_features,
                               output_dim=150,
                               input_length=MAX_SEQUENCE_LENGTH))
    model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(64, recurrent_dropout=0.5, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=["accuracy"])
    return model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.4)))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.003)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.hs', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=10, callbacks=[es, mc], batch_size=32, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test).round()

In [ ]:
y_pred

# **Evaluation**

In [ ]:
train_accuracy = round(metrics.accuracy_score(y_train, model.predict(X_train).round()) * 100)
train_accuracy

In [ ]:
print('Accuracy  is  : ', (metrics.accuracy_score(y_test, y_pred)))
print('Recall  is    : ', (metrics.recall_score(y_test, y_pred)))
print('Precision  is : ', (metrics.precision_score(y_test, y_pred)))

In [ ]:
conm = confusion_matrix(y_test,y_pred)
conm

In [ ]:
plt.figure(figsize=(7, 5))
sns.heatmap(conm, annot=True, fmt='d', cmap='cool')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

# **Submission**

In [ ]:
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
test_data.head().style.background_gradient(cmap='coolwarm')

In [ ]:
test_data['clean_text'] = test_data["text"].apply(clean_txt)
test_data['clean_text'].head()

In [ ]:
l =50
max_features=5000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data['clean_text'].values)
X = tokenizer.texts_to_sequences(train_data['clean_text'].values)
X = pad_sequences(X, maxlen =l)

In [ ]:
tokenizer.fit_on_texts(train_data['clean_text'].values)
test_token = tokenizer.texts_to_sequences(test_data['clean_text'].values)
test_token = pad_sequences(test_token, maxlen =l)

In [ ]:
embed_dim = 128
lstm_out = 128

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.4)))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.003)

model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X,y, epochs = 10,validation_split = 0.2 ,callbacks=[es_callback], batch_size=32)

In [ ]:
y_hat = model.predict(test_token).round()
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission['target'] = np.round(y_hat).astype('int')
submission.to_csv('submission.csv', index=False)
submission.describe().style.background_gradient(cmap='coolwarm')